# 파이썬 코딩의 기술 - ch.3 함수

## Better way 19. 함수가 여러 값을 반환하는 경우 절대로 네 값 이상을 언패킹하지 말라

In [ ]:
param = 3, 6, 2, 1, 4, 6, 8, 2, 5

# wrong
def get_stats1(numbers):
    minimum = min(numbers)
    maximum = max(numbers)
    count = len(numbers)
    average = sum(numbers) / count
    return minimum, maximum, count, average

minimum, maximum, count, average = get_stats1(param)
print(minimum, maximum, count, average)

print('============')
# instead
from collections import namedtuple
def get_stats2(numbers):
    Result = namedtuple('Result', ['minimum', 'maximum', 'count', 'average'])
    minimum = min(numbers)
    maximum = max(numbers)
    count = len(numbers)
    average = sum(numbers) / count
    return Result(minimum, maximum, count, average)

result = get_stats2(param)
print(result.minimum, result.maximum, result.count, result.average)

1 8 9 4.111111111111111
1 8 9 4.111111111111111


## Better way 20. None을 반환하기보다는 예외를 발생시켜라

In [ ]:
# wrong
def careful_divide1(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None
print(careful_divide1(5, 0))

# right
def careful_divide2(a, b):
    """ a를 b로 나눈다.

    Raises:
        ValueError : b가 0이어서 나눗셈을 할 수 없을 때
    """
    try : 
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('잘못된 입력')
print(careful_divide2(5, 0))

None


ValueError: ignored

## Better way 21. 변수 영역과 클로저의 상호작용 방식을 이해하라

 파이썬은 다음 순서로 참조를 진행한다.
 1. 현재 함수의 영역
 2. 현재 함수를 둘러싼 영역
 3. 현재 코드가 들어 있는 모듈의 영역 (전역 영역, global scope)
 4. 내장 영역 (len, str 등의 함수가 들어 있는 영역)
 
 앞 순서에서 변수를 찾으면 상위 스코프로 올라가지 않는다.
 여기서 발생한 문제를 <u>영역 지정 버그(scoping bug)</u>라고 한다.

 이를 해결하기 위해 nonlocal 키워드를 활용한다.

In [ ]:
kw = 5
numbers = [2, 3, 7, 4, 6, 5, 9, 2, 6]

# wrong
def is_in1(kw, numbers):
    found = False
    def helper(x):
        if x in numbers:
            found = True
    helper(kw)
    return found

print(is_in1(kw, numbers))

print('============')
# right
def is_in2(kw, numbers):
    found = False
    def helper(x):
        nonlocal found
        if x in numbers:
            found = True
    helper(kw)
    return found

print(is_in2(kw, numbers))

False
True


## Better way 22. 변수 위치 인자를 사용해 시각적인 잡음을 줄여라

In [ ]:
def log1(message, *values):
    if not values:
        print(message)
    else:
        value_str = ', '.join(str(value) for value in values)
        print(f'{message} : {value_str}')

log1('오류')
log1('오류', 256, 64, 1024)

오류
오류 : 256, 64, 1024


## Better way 23. 키워드 인자로 선택적인 기능을 제공하라

In [ ]:
# example 1
def log2(message, code, call, ping):
    print(f'{message}: code {code}, call {call}, ping {ping}')

values = {
    'code': '404',
    'call': '8000',
    'ping': '20ms'
}
log2('오류 메시지', **values)

print('============')
# example 2
def log3(**kwargs):
    kwargs_str = ''
    for key, value in kwargs.items():
        kwargs_str += f'{key}: {value}, '
    print(kwargs_str)
log3(message='오류메시지', code='404', ping='20ms')
log3(**values)

오류 메시지: code 404, call 8000, ping 20ms
message: 오류메시지, code: 404, ping: 20ms, 
code: 404, call: 8000, ping: 20ms, 


## Better way 24. None과 독스트링을 사용해 동적인 디폴트 인자를 지정하라

디폴트 인자는 함수가 정의되는 시점에 단 한 번 정의되기 때문에 동적인 정보는 담기지 않음을 명심하라.

In [ ]:
import json

# wrong
def decode1(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

foo = decode1('잘못된 데이터')
foo['stuff'] = 5
bar = decode1('잘못된 데이터 2')
bar['meep'] = 1
print('Foo :', foo)
print('Bar :', bar)

print('============')
def decode2(data, default=None):
    """문자열로부터 JSON 데이터를 읽어온다

    Args:
        data: 디코딩할 json 데이터.
        default: 디코딩 실패 시 반환할 값이다.
            디폴트 값은 빈 딕셔너리다.
    """
    try:
        return json.loads(data)
    except ValueError:
        if default is None:
            default = {}
        return default

foo = decode2('잘못된 데이터')
foo['stuff'] = 5
bar = decode2('잘못된 데이터 2')
bar['meep'] = 1
print('Foo :', foo)
print('Bar :', bar)

Foo : {'stuff': 5, 'meep': 1}
Bar : {'stuff': 5, 'meep': 1}
Foo : {'stuff': 5}
Bar : {'meep': 1}


## Better way 25. 위치로만 인자를 지정하게 하거나 키워드로만 인자를 지정하게 해서 함수 호출을 명확하게 하라

parameter에 '*' 기호 : 위치 인자의 마지막과 키워드만 사용하는 시작의 구분

-> 위치 인자는 키워드로 호출이 가능하지만, 키워드 인자는 위치 인자로 호출 불가능

parameter에 '/' 기호 : 위치로만 지정하는 인자의 끝을 표시

-> 위치 인자를 키워드로 사용이 불가능

\* 파이썬 3.8 이후로만 지원

In [11]:
def safe_devision1(number, divisor, *,
                   ignore_overflow=False,
                   throw_exception=False):
    ...

try:
    safe_devision1(4, 0, True)
except Exception as e:
    print('error :', e)

def safe_devision2(number, divisor, /, *,
                   ignore_overflow=False,
                   throw_exception=False):
    ...

try:
    safe_devision2(4, divisor=0, ignore_overflow=True)
except Exception as e:
    print('error :', e)

error : safe_devision1() takes 2 positional arguments but 3 were given
error : safe_devision2() got some positional-only arguments passed as keyword arguments: 'divisor'


## Better way 26. functools.wrap을 사용해 함수 데코레이터를 정의하라

데코레이터를 직접 만들 때 고려해야 하는 여러 점을 구현해준다.

ex) 중요 메타데이터; 함수의 이름, help(func) 등

In [13]:
# wrong
def trace1(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) -> {result!r}')
        return result
    return wrapper

# right
from functools import wraps

def trace2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) -> {result!r}')
        return result
    return wrapper

# result
@trace1
def hello1(*args, **kwargs):
    pass

@trace2
def hello2(*args, **kwargs):
    pass

print(hello1)
print(hello2)

<function trace1.<locals>.wrapper at 0x7f8a72c81af0>
<function hello2 at 0x7f8a72c81dc0>
